In [2]:
from keras.datasets import cifar10
from sklearn.preprocessing import StandardScaler
from keras.applications.vgg16 import VGG16
from keras.models import Model
import numpy as np
from skimage import feature, color
from sklearn import preprocessing
from cvxopt import matrix, solvers
from matplotlib import pyplot as plt

In [3]:
# load cifar10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# normalize data
y_train = y_train.ravel()
y_test = y_test.ravel()

# extract features
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
model = Model(inputs=base_model.input, outputs=base_model.output)
features_train = model.predict(x_train)
features_test = model.predict(x_test)

# reshape features
features_train = features_train.reshape(features_train.shape[0], -1)
features_test = features_test.reshape(features_test.shape[0], -1)

# standardize features
scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)

print("Training features shape:", features_train.shape)
print("Testing features shape:", features_test.shape)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 38s 24ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step
Training features shape: (50000, 512)
Testing features shape: (10000, 512)


In [27]:
def hinge_loss(y_true, y_pred):
    return np.maximum(0, 1 - y_true * y_pred).mean()

def accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

In [26]:
class KernelSVM:
    def __init__(self, kernel='linear', C=1.0):
        self.kernel = kernel
        self.C = C
        self.alpha = None
        self.support_vectors = None
        self.support_vector_labels = None
        self.b = 0

    def kernel_function(self, x, y):
        if self.kernel == 'linear':
            return np.dot(x, y)
        elif self.kernel == 'polynomial':
            return (1 + np.dot(x, y)) ** 3
        elif self.kernel == 'rbf':
            sigma = 1.0
            return np.exp(-np.linalg.norm(x-y)**2 / (2 * (sigma ** 2)))
        else:
            raise ValueError("Unknown kernel function.")

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y = y.reshape(-1, 1) * 1.0
        X_dash = y * X
        H = np.dot(X_dash , X_dash.T) * 1.

        # Convert into cvxopt format
        P = matrix(H)
        q = matrix(-np.ones((n_samples, 1)))
        G = matrix(np.vstack((np.eye(n_samples)*-1,np.eye(n_samples))))
        h = matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * self.C)))
        A = matrix(y.reshape(1, -1))
        b = matrix(np.zeros(1))

        # Solve QP problem
        solution = solvers.qp(P, q, G, h, A, b)

        # Extract support vectors
        alphas = np.array(solution['x'])
        self.alpha = alphas[alphas > 1e-4].flatten()
        sv = alphas > 1e-4
        ind = np.where(sv)[0]
        self.support_vectors = X[ind]
        self.support_vector_labels = y[ind]
        self.b = np.mean(y[ind] - np.dot(self.support_vectors, self.support_vectors.T).dot(alphas[sv] * y[sv]))
        
        # Calculate the predictions on the training set
        y_pred = self.predict(X)
        # Convert predictions back to original class labels
        y_pred = np.where(y_pred < 0, -1, 1)
        y = np.where(y <= 0, -1, 1)  # Ensure y is in {-1, 1}
        
        # Calculate the loss and accuracy
        loss = hinge_loss(y, y_pred)
        acc = accuracy(y, y_pred)
        
        return loss, acc

    def predict(self, X):
        y_predict = np.zeros(len(X))
        for i in range(len(X)):
            prediction = 0
            for alpha, sv_y, sv in zip(self.alpha, self.support_vector_labels, self.support_vectors):
                prediction += alpha * sv_y * self.kernel_function(X[i], sv)
            prediction += self.b
            y_predict[i] = np.sign(prediction).item()
        return y_predict

In [29]:
# sub-sample the training set
indices = np.random.choice(range(features_train.shape[0]), size=10000, replace=False)
sub_features_train = features_train[indices]
sub_y_train = y_train[indices]


svm_models = []
num_classes = 10
for class_idx in range(num_classes):
    print(f"Training model for class {class_idx}...")
    
    # Convert the labels to binary
    binary_y_train = np.where(sub_y_train == class_idx, 1, -1)
    
    # Train the model
    svm_model = KernelSVM(kernel='linear', C=1.0)
    loss, acc = svm_model.fit(sub_features_train, binary_y_train)
    
    # Store the model
    svm_models.append(svm_model)
    
    print(f"Training Loss: {loss:.4f}, Training Accuracy: {acc:.4f}")


Training model for class 0...
     pcost       dcost       gap    pres   dres
 0: -2.0634e+03 -2.5220e+04  2e+05  3e+00  4e-12
 1: -1.3639e+03 -1.5721e+04  3e+04  4e-01  3e-12
 2: -1.1957e+03 -7.2551e+03  1e+04  1e-01  2e-12
 3: -1.1246e+03 -4.7236e+03  5e+03  6e-02  2e-12
 4: -1.0907e+03 -3.5638e+03  4e+03  4e-02  2e-12
 5: -1.0840e+03 -3.1660e+03  3e+03  3e-02  2e-12
 6: -1.0381e+03 -2.8883e+03  2e+03  2e-02  2e-12
 7: -1.0712e+03 -2.0447e+03  1e+03  7e-03  2e-12
 8: -1.1042e+03 -1.6416e+03  6e+02  2e-03  2e-12
 9: -1.1290e+03 -1.4744e+03  4e+02  8e-04  2e-12
10: -1.1488e+03 -1.3970e+03  3e+02  5e-04  2e-12
11: -1.1655e+03 -1.3310e+03  2e+02  1e-04  2e-12
12: -1.1831e+03 -1.2879e+03  1e+02  6e-05  2e-12
13: -1.1937e+03 -1.2652e+03  7e+01  3e-05  2e-12
14: -1.2039e+03 -1.2459e+03  4e+01  1e-05  2e-12
15: -1.2077e+03 -1.2373e+03  3e+01  5e-06  2e-12
16: -1.2125e+03 -1.2299e+03  2e+01  2e-06  2e-12
17: -1.2151e+03 -1.2260e+03  1e+01  6e-07  2e-12
18: -1.2173e+03 -1.2231e+03  6e+00  2e-0

In [30]:
# initialize decision function values
decision_function_values = np.zeros((features_test.shape[0], num_classes))

# use each SVM model to predict the decision function values
for class_idx, svm_model in enumerate(svm_models):
    print(f"Predicting with model for class {class_idx}...")
    decision_values = svm_model.predict(features_test)
    decision_function_values[:, class_idx] = decision_values

# select the class with the maximum decision function value
y_pred = np.argmax(decision_function_values, axis=1)

# calculate the test accuracy
test_accuracy = accuracy(y_test, y_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")


Predicting with model for class 0...
Predicting with model for class 1...
Predicting with model for class 2...
Predicting with model for class 3...
Predicting with model for class 4...
Predicting with model for class 5...
Predicting with model for class 6...
Predicting with model for class 7...
Predicting with model for class 8...
Predicting with model for class 9...
Test Accuracy: 0.4681


In [43]:
indices = np.random.choice(range(features_train.shape[0]), size=10000, replace=False)
sub_features_train = features_train[indices]
sub_y_train = y_train[indices]

In [45]:
# train a linear SVM model using the sklearn library on original features
from sklearn.svm import SVC
svm = SVC(kernel='linear', C=1.0)
svm.fit(features_train, y_train)

# get the training accuracy
y_pred_train = svm.predict(features_train)
train_accuracy = accuracy(y_train, y_pred_train)
print(f"Training Accuracy (Original Features): {train_accuracy:.4f}")

# get the test accuracy
y_pred_test = svm.predict(features_test)
test_accuracy = accuracy(y_test, y_pred_test)
print(f"Test Accuracy (Original Features): {test_accuracy:.4f}")

Training Accuracy (Original Features): 0.6799
Test Accuracy (Original Features): 0.6095


In [40]:
svm_models = []
num_classes = 10
for class_idx in range(num_classes):
    print(f"Training model for class {class_idx}...")
    
    # Convert the labels to binary
    binary_y_train = np.where(sub_y_train == class_idx, 1, -1)
    
    # Train the model
    svm_model = KernelSVM(kernel='polynomial', C=10)
    loss, acc = svm_model.fit(sub_features_train, binary_y_train)
    
    # Store the model
    svm_models.append(svm_model)
    
    print(f"Training Loss: {loss:.4f}, Training Accuracy: {acc:.4f}")

# initialize decision function values
decision_function_values = np.zeros((features_test.shape[0], num_classes))

# use each SVM model to predict the decision function values
for class_idx, svm_model in enumerate(svm_models):
    print(f"Predicting with model for class {class_idx}...")
    decision_values = svm_model.predict(features_test)
    decision_function_values[:, class_idx] = decision_values

# select the class with the maximum decision function value
y_pred = np.argmax(decision_function_values, axis=1)

# calculate the test accuracy
test_accuracy = accuracy(y_test, y_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

Training model for class 0...
     pcost       dcost       gap    pres   dres
 0: -3.7871e+02 -4.7591e+04  1e+05  6e-01  1e-12
 1: -8.7357e+01 -1.6363e+04  3e+04  1e-01  1e-12
 2:  7.1424e+01 -2.8786e+03  5e+03  1e-02  6e-13
 3:  3.4034e+01 -4.3417e+02  7e+02  2e-03  1e-13
 4:  1.4387e+01 -1.0893e+02  2e+02  4e-04  3e-14
 5:  3.1241e+00 -8.4993e+00  1e+01  2e-06  2e-14
 6: -2.9156e-01 -2.0520e+00  2e+00  1e-07  8e-15
 7: -7.0476e-01 -1.4958e+00  8e-01  4e-08  5e-15
 8: -9.2682e-01 -1.2625e+00  3e-01  5e-09  5e-15
 9: -1.0051e+00 -1.1178e+00  1e-01  1e-09  6e-15
10: -1.0382e+00 -1.0567e+00  2e-02  2e-16  6e-15
11: -1.0447e+00 -1.0461e+00  1e-03  2e-16  5e-15
12: -1.0452e+00 -1.0452e+00  3e-05  2e-16  5e-15
13: -1.0452e+00 -1.0452e+00  6e-07  2e-16  6e-15
Optimal solution found.
Training Loss: 0.8048, Training Accuracy: 0.5976
Training model for class 1...
     pcost       dcost       gap    pres   dres
 0: -6.5386e+02 -6.1069e+04  2e+05  8e-01  2e-12
 1: -2.4507e+02 -2.4188e+04  5e+04  

In [41]:
svm_models = []
num_classes = 10
for class_idx in range(num_classes):
    print(f"Training model for class {class_idx}...")
    
    # Convert the labels to binary
    binary_y_train = np.where(sub_y_train == class_idx, 1, -1)
    
    # Train the model
    svm_model = KernelSVM(kernel='rbf', C=0.01)
    loss, acc = svm_model.fit(sub_features_train, binary_y_train)
    
    # Store the model
    svm_models.append(svm_model)
    
    print(f"Training Loss: {loss:.4f}, Training Accuracy: {acc:.4f}")

# initialize decision function values
decision_function_values = np.zeros((features_test.shape[0], num_classes))

# use each SVM model to predict the decision function values
for class_idx, svm_model in enumerate(svm_models):
    print(f"Predicting with model for class {class_idx}...")
    decision_values = svm_model.predict(features_test)
    decision_function_values[:, class_idx] = decision_values

# select the class with the maximum decision function value
y_pred = np.argmax(decision_function_values, axis=1)

# calculate the test accuracy
test_accuracy = accuracy(y_test, y_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")

Training model for class 0...
     pcost       dcost       gap    pres   dres
 0: -3.8630e+01 -1.7953e+01  6e+03  8e+01  1e-13
 1: -2.0013e+00 -1.7733e+01  2e+02  2e+00  1e-13
 2: -9.3555e-01 -1.3308e+01  3e+01  2e-01  1e-14
 3: -6.4153e-01 -5.2196e+00  7e+00  4e-02  4e-15
 4: -5.4011e-01 -1.7706e+00  2e+00  8e-03  2e-15
 5: -5.7599e-01 -9.3022e-01  4e-01  2e-03  2e-15
 6: -6.0903e-01 -7.1187e-01  1e-01  2e-04  2e-15
 7: -6.2894e-01 -6.5497e-01  3e-02  6e-17  2e-15
 8: -6.3704e-01 -6.3997e-01  3e-03  6e-17  2e-15
 9: -6.3813e-01 -6.3829e-01  2e-04  6e-17  2e-15
10: -6.3819e-01 -6.3819e-01  2e-06  7e-17  2e-15
11: -6.3819e-01 -6.3819e-01  3e-08  6e-17  2e-15
Optimal solution found.
Training Loss: 0.1800, Training Accuracy: 0.9100
Training model for class 1...
     pcost       dcost       gap    pres   dres
 0: -6.4610e+01 -1.9386e+01  7e+03  8e+01  1e-13
 1: -2.5517e+00 -1.9172e+01  1e+02  1e+00  2e-13
 2: -1.3997e+00 -1.3444e+01  2e+01  2e-01  2e-14
 3: -1.0380e+00 -5.7152e+00  7e+00  